# <div align="center" style="color: #ff5733;">PSI Monitoring</div>

# Declare Libraries

In [1]:
# %% [markdown]
# # Jupyter Notebook Loading Header
#
# This is a custom loading header for Jupyter Notebooks in Visual Studio Code.
# It includes common imports and settings to get you started quickly.

# %% [markdown]
## Import Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from google.cloud import bigquery
import os
path = r'C:\Users\DwaipayanChakroborti\AppData\Roaming\gcloud\legacy_credentials\dchakroborti@tonikbank.com\adc.json'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = path
client = bigquery.Client(project='prj-prod-dataplatform')

# %% [markdown]
## Configure Settings
# Set options or configurations as needed
# Example: pd.set_option('display.max_columns', None)

# s_apps_score

In [55]:
sq = """
with base as 
(select 
  a.digitalLoanAccountId, 
  FORMAT_DATE('%Y-%m', a.ln_appln_submit_datetime) Application_month,
  FORMAT_DATE('%F', DATE_TRUNC(a.ln_appln_submit_datetime, WEEK(MONDAY))) as Appl_week_start_date,
  EXTRACT(WEEK(MONDAY) FROM a.ln_appln_submit_datetime) as Appl_week_number,
  a.ln_loan_type,
  case when date_trunc(a.ln_appln_submit_datetime, day) between '2023-07-01' and '2024-06-30' then 'Train'
       when date_trunc(a.ln_appln_submit_datetime, day) >= '2024-08-01' then 'Test'
       Else 'Other' end dataselection,
  a.s_apps_score,
 from 
 prj-prod-dataplatform.risk_mart.sil_risk_ds_master_20230101_20250206 a
 where a.ln_loan_applied_flag = 1 and ln_dl_rule_reject_flag = 0
 and date_trunc(a.ln_appln_submit_datetime, day) >= '2023-04-01'
 )
 select * from base where s_apps_score is not null and dataselection in ('Train', 'Test');"""
 
df = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The shape of df before duplicate drop is:\t {df.shape}")

df = df.drop_duplicates(keep='first')

print(f"The shape of df after duplicate drop is:\t {df.shape}")

Job ID 2acfb3f2-738e-4e55-b6e3-dd978a63363f successfully executed: 100%|██████████|

d:\Anaconda3\envs\inc_env\lib\site-packages\google\cloud\bigquery\table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



Downloading: 100%|██████████|
The shape of df before duplicate drop is:	 (263969, 7)
The shape of df after duplicate drop is:	 (263969, 7)


In [34]:
df.head()

,digitalLoanAccountId,Application_month,Appl_week_start_date,Appl_week_number,ln_loan_type,dataselection,s_apps_score
0,1f0d420f-746d-4f58-a4e7-50c345d701db,2023-10,2023-10-16,42,SIL-Instore,Train,0.641338
1,3632ebdd-470d-40e5-8de4-c1b807c981ab,2023-08,2023-08-07,32,SIL-Instore,Train,0.582026
2,b8d7faed-c257-4735-96bb-529eb556a1e9,2023-09,2023-09-18,38,SIL-Instore,Train,0.326905
3,065cf333-9d42-428d-92da-a7c8d18e57ea,2023-10,2023-10-09,41,SIL-Instore,Train,0.625680
4,adb459b3-9e99-41b0-a708-4914f4892188,2023-09,2023-09-18,38,SIL-Instore,Train,0.471751


In [32]:
# import pandas as pd
# import numpy as np
# import matplotlib.pyplot as plt
# import seaborn as sns
# from scipy import stats

# def calculate_psi(train_dist, test_dist):
#     """
#     Calculate PSI for a single comparison between training and test distributions
#     """
#     psi_value = 0
    
#     # Add small epsilon to avoid division by zero or log of zero
#     epsilon = 1e-6
    
#     for i in range(len(train_dist)):
#         actual = train_dist[i] + epsilon
#         expected = test_dist[i] + epsilon
        
#         if actual != 0 and expected != 0:
#             psi_value += (actual - expected) * np.log(actual / expected)
    
#     return psi_value

# def calculate_monthly_psi(data):
#     """
#     Calculate PSI for each test month compared to the entire training period
#     """
#     # Ensure Application_month is datetime
#     if isinstance(data['Application_month'].iloc[0], str):
#         data['Application_month'] = pd.to_datetime(data['Application_month'].astype(str) + '-01')
    
#     # Get training period data
#     train_data = data[data['dataselection'] == 'Train']['s_apps_score'].values
    
#     # Create decile bins based on training data
#     num_bins = 10
#     bins = np.percentile(train_data, np.linspace(0, 100, num_bins + 1))
    
#     # Ensure unique bin edges
#     bins = np.unique(bins)
#     if len(bins) < num_bins + 1:
#         # If we have duplicate edges, create evenly spaced bins
#         bins = np.linspace(train_data.min(), train_data.max(), num_bins + 1)
    
#     # Calculate baseline (training) distribution
#     train_hist, _ = np.histogram(train_data, bins=bins)
#     train_dist = train_hist / len(train_data)
    
#     # Calculate PSI for each test month
#     test_data = data[data['dataselection'] == 'Test']
#     monthly_psi = {}
    
#     for month in sorted(test_data['Application_month'].unique()):
#         month_data = test_data[test_data['Application_month'] == month]['s_apps_score'].values
        
#         if len(month_data) > 0:  # Only calculate if we have data for this month
#             test_hist, _ = np.histogram(month_data, bins=bins)
#             test_dist = test_hist / len(month_data)
            
#             psi_value = calculate_psi(test_dist, train_dist)
#             monthly_psi[month] = psi_value
    
#     return monthly_psi, bins

# def plot_psi_results(monthly_psi, data, bins):
#     """
#     Create visualizations for PSI analysis
#     """
#     plt.figure(figsize=(15, 6))
    
#     # Plot 1: Monthly PSI Values
#     plt.subplot(1, 2, 1)
#     months = list(monthly_psi.keys())
#     psi_values = list(monthly_psi.values())
    
#     plt.plot(months, psi_values, marker='o')
#     plt.title('Monthly Population Stability Index')
#     plt.xlabel('Month')
#     plt.ylabel('PSI Value')
#     plt.grid(True)
#     plt.xticks(rotation=45)
    
#     # Add reference lines
#     plt.axhline(y=0.1, color='y', linestyle='--', label='Minor Shift (0.1)')
#     plt.axhline(y=0.2, color='r', linestyle='--', label='Major Shift (0.2)')
#     plt.legend()
    
#     # Plot 2: Distribution Comparison
#     plt.subplot(1, 2, 2)
    
#     # Training distribution
#     train_data = data[data['dataselection'] == 'Train']['s_apps_score']
#     plt.hist(train_data, bins=bins, density=True, alpha=0.5, label='Training', color='blue')
    
#     # Latest test month distribution
#     test_data = data[data['dataselection'] == 'Test']
#     if not test_data.empty:
#         latest_month = max(test_data['Application_month'])
#         latest_test_data = test_data[test_data['Application_month'] == latest_month]['s_apps_score']
#         plt.hist(latest_test_data, bins=bins, density=True, alpha=0.5, 
#                 label=f'Test ({latest_month.strftime("%Y-%m")})', color='red')
    
#     plt.title('Score Distribution Comparison')
#     plt.xlabel('Score')
#     plt.ylabel('Density')
#     plt.legend()
    
#     plt.tight_layout()
#     plt.show()

# def print_summary_statistics(data):
#     """
#     Print summary statistics for both training and test periods
#     """
#     print("\nSummary Statistics:")
#     print("\nTraining Period:")
#     train_stats = data[data['dataselection'] == 'Train']['s_apps_score'].describe()
#     print(train_stats)
    
#     print("\nTest Period:")
#     test_stats = data[data['dataselection'] == 'Test']['s_apps_score'].describe()
#     print(test_stats)

# def analyze_psi(df):
#     """
#     Main analysis function
#     """
#     # Calculate monthly PSI
#     monthly_psi, bins = calculate_monthly_psi(df)
    
#     # Print PSI values
#     print("\nMonthly PSI Values:")
#     for month, psi in sorted(monthly_psi.items()):
#         # Fixed string formatting
#         print(f"{month.strftime('%Y-%m')}: {psi:.4f}")
    
#     # Plot results
#     plot_psi_results(monthly_psi, df, bins)
    
#     # Print summary statistics
#     print_summary_statistics(df)

# # Example usage
# def main():
#     # Read your data into a DataFrame
#     # Example of how to create DataFrame from your SQL query results
#     # df = pd.DataFrame({
#     #     'digitalLoanAccountId': [],
#     #     'Application_month': [],
#     #     'dataselection': [],
#     #     's_apps_score': []
#     # })
    
#     # Run the analysis
#     analyze_psi(df)

# if __name__ == "__main__":
#     main()

In [33]:
# import pandas as pd
# import numpy as np
# import matplotlib.pyplot as plt
# import seaborn as sns
# from scipy import stats

# def calculate_psi(train_dist, test_dist):
#     """
#     Calculate PSI for a single comparison between training and test distributions
#     """
#     psi_value = 0
#     epsilon = 1e-6
    
#     for i in range(len(train_dist)):
#         actual = train_dist[i] + epsilon
#         expected = test_dist[i] + epsilon
        
#         if actual != 0 and expected != 0:
#             psi_value += (actual - expected) * np.log(actual / expected)
    
#     return psi_value

# def calculate_monthly_psi(data):
#     """
#     Calculate PSI for each test month compared to the entire training period
#     """
#     # Ensure Application_month is datetime
#     if isinstance(data['Application_month'].iloc[0], str):
#         data['Application_month'] = pd.to_datetime(data['Application_month'].astype(str) + '-01')
    
#     # Get training period data
#     train_data = data[data['dataselection'] == 'Train']['s_apps_score'].values
    
#     # Create decile bins based on training data
#     num_bins = 10
#     bins = np.percentile(train_data, np.linspace(0, 100, num_bins + 1))
    
#     # Ensure unique bin edges
#     bins = np.unique(bins)
#     if len(bins) < num_bins + 1:
#         bins = np.linspace(train_data.min(), train_data.max(), num_bins + 1)
    
#     # Calculate baseline (training) distribution
#     train_hist, _ = np.histogram(train_data, bins=bins)
#     train_dist = train_hist / len(train_data)
    
#     # Calculate PSI for each test month
#     test_data = data[data['dataselection'] == 'Test']
#     monthly_psi = {}
    
#     for month in sorted(test_data['Application_month'].unique()):
#         month_data = test_data[test_data['Application_month'] == month]['s_apps_score'].values
        
#         if len(month_data) > 0:
#             test_hist, _ = np.histogram(month_data, bins=bins)
#             test_dist = test_hist / len(month_data)
            
#             psi_value = calculate_psi(test_dist, train_dist)
#             monthly_psi[month] = psi_value
    
#     return monthly_psi, bins

# def create_psi_dataframe(monthly_psi, loan_type):
#     """
#     Create a structured DataFrame from PSI results
#     """
#     # Create DataFrame from monthly PSI dictionary
#     df_psi = pd.DataFrame([
#         {
#             'Dated': month,
#             'Score_Category': f'Apps_Score_{month.strftime("%Y-%m")}',
#             'Loan_Product': loan_type,
#             'Date_Category': 'Monthly',
#             'psi': psi_value
#         }
#         for month, psi_value in monthly_psi.items()
#     ])
    
#     # Sort by date
#     df_psi = df_psi.sort_values('Dated').reset_index(drop=True)
    
#     return df_psi

# def plot_psi_results(df_psi):
#     """
#     Create visualizations for PSI analysis using the structured DataFrame
#     """
#     plt.figure(figsize=(15, 6))
    
#     # Plot Monthly PSI Values
#     plt.plot(df_psi['Dated'], df_psi['psi'], marker='o')
#     plt.title('Monthly Population Stability Index')
#     plt.xlabel('Month')
#     plt.ylabel('PSI Value')
#     plt.grid(True)
#     plt.xticks(rotation=45)
    
#     # Add reference lines
#     plt.axhline(y=0.1, color='y', linestyle='--', label='Minor Shift (0.1)')
#     plt.axhline(y=0.2, color='r', linestyle='--', label='Major Shift (0.2)')
#     plt.legend()
    
#     plt.tight_layout()
#     plt.show()

# def analyze_psi(df, loan_type='SIL'):
#     """
#     Main analysis function that returns structured DataFrame
#     """
#     # Calculate monthly PSI
#     monthly_psi, bins = calculate_monthly_psi(df)
    
#     # Create structured DataFrame
#     df_psi = create_psi_dataframe(monthly_psi, loan_type)
    
#     # Display results
#     print("\nPSI Analysis Results:")
#     print(df_psi.to_string(index=False))
    
#     # Plot results
#     plot_psi_results(df_psi)
    
#     return df_psi

# # Example usage
# def main():
#     # Read your data into a DataFrame
#     # Example of how to structure your data
#     # df = pd.DataFrame({
#     #     'digitalLoanAccountId': [],
#     #     'Application_month': [],
#     #     'dataselection': [],
#     #     's_apps_score': [],
#     #     'ln_loan_type': []  # Add loan type if available in your data
#     # })
    
#     # Run the analysis
#     df_psi = analyze_psi(df, loan_type='SIL')
    
#     # Export to CSV if needed
#     # df_psi.to_csv('psi_results.csv', index=False)

# if __name__ == "__main__":
#     main()

In [34]:
# df_psi = analyze_psi(df, loan_type='SIL')
# df_psi.head()

In [35]:
# import pandas as pd
# import numpy as np
# from datetime import datetime

# # Assuming your DataFrame is called 'df' and has the structure from the image
# # Replace this with your actual DataFrame loading process
# sq = """
# with base as 
# (select 
#   a.digitalLoanAccountId, 
#   FORMAT_DATE('%Y-%m', a.ln_appln_submit_datetime) Application_month,
#   FORMAT_DATE('%F', DATE_TRUNC(a.ln_appln_submit_datetime, WEEK(MONDAY))) as Appl_week_start_date,
#   EXTRACT(WEEK(MONDAY) FROM a.ln_appln_submit_datetime) as Appl_week_number,
#   a.ln_loan_type,
#   case when date_trunc(a.ln_appln_submit_datetime, day) between '2023-07-01' and '2024-07-01' then 'Train'
#        Else 'Test' end dataselection,
#   a.s_apps_score,
#  from 
#  prj-prod-dataplatform.risk_mart.sil_risk_ds_master_20230101_20250206 a
#  where a.ln_loan_applied_flag = 1
#  and date_trunc(a.ln_appln_submit_datetime, day) >= '2023-06-01'
#  )
#  select * from base where s_apps_score is not null;"""
 
# df = client.query(sq).to_dataframe(progress_bar_type='tqdm')


# # Convert Application_month to datetime if it's not already
# if df['Application_month'].dtype != 'datetime64[ns]':
#     df['Application_month'] = pd.to_datetime(df['Application_month'] + '-01')

# # Separate train and test data
# train_df = df[df['dataselection'] == 'Train']
# test_df = df[df['dataselection'] == 'Test']

# # Calculate decile bins for the entire training set
# train_deciles = pd.qcut(train_df['s_apps_score'], 10, labels=False, retbins=True)
# train_decile_bins = train_deciles[1]

# # Function to calculate PSI using the pre-defined decile bins
# def calculate_psi_with_bins(data_scores, decile_bins):
#     """Calculates PSI using pre-defined decile bins."""
#     data_deciles = pd.cut(data_scores, bins=decile_bins, labels=False, include_lowest=True)
#     distribution = pd.Series(data_deciles).value_counts().sort_index() / len(data_scores)

#     # Align with training distribution
#     all_bins = range(10)  # Assuming 10 deciles
#     distribution_aligned = distribution.reindex(all_bins, fill_value=0)
#     train_dist_aligned = pd.Series(train_deciles[0]).value_counts().sort_index() / len(train_df['s_apps_score'])
#     train_dist_aligned = train_dist_aligned.reindex(all_bins, fill_value=0)

#     psi_values = (distribution_aligned - train_dist_aligned) * np.log(distribution_aligned / train_dist_aligned)
#     return psi_values.sum()

# # Calculate PSI for the entire training set
# train_psi = calculate_psi_with_bins(train_df['s_apps_score'], train_decile_bins)

# # Calculate monthly PSI for the test set
# monthly_psi_results = []
# for month in sorted(test_df['Application_month'].unique()):
#     month_str = month.strftime('%Y-%m')
#     month_df = test_df[test_df['Application_month'] == month]
#     if not month_df.empty:
#         month_psi = calculate_psi_with_bins(month_df['s_apps_score'], train_decile_bins)
#         monthly_psi_results.append({
#             'Month': month_str,
#             'loan_type': month_df['ln_loan_type'].iloc[0],  # Assuming loan_type is consistent in the month
#             'scorename': 's_apps_score',
#             'DateCategory': 'Monthly',
#             'psivalues': month_psi
#         })

# # Create the output DataFrame
# output_df = pd.DataFrame(monthly_psi_results)
# output_df.head(10)

In [35]:
import pandas as pd
import numpy as np
from datetime import datetime

# Assuming your DataFrame is called 'df' and has the structure from the image
# Replace this with your actual DataFrame loading process
sq = """
with base as 
(select 
  a.digitalLoanAccountId, 
  FORMAT_DATE('%Y-%m', a.ln_appln_submit_datetime) Application_month,
  FORMAT_DATE('%F', DATE_TRUNC(a.ln_appln_submit_datetime, WEEK(MONDAY))) as Appl_week_start_date,
  EXTRACT(WEEK(MONDAY) FROM a.ln_appln_submit_datetime) as Appl_week_number,
  a.ln_loan_type,
  case when date_trunc(a.ln_appln_submit_datetime, day) between '2023-07-01' and '2024-06-30' then 'Train'
       when date_trunc(a.ln_appln_submit_datetime, day) >= '2024-07-01' then 'Test'
       Else 'Other' end dataselection,
  a.s_apps_score,
 from 
 prj-prod-dataplatform.risk_mart.sil_risk_ds_master_20230101_20250206 a
 where a.ln_loan_applied_flag = 1 and ln_dl_rule_reject_flag = 0
 and date_trunc(a.ln_appln_submit_datetime, day) >= '2023-04-01'
 )
 select * from base where s_apps_score is not null and dataselection in ('Train', 'Test');"""
 
df = client.query(sq).to_dataframe(progress_bar_type='tqdm')

# Convert Application_month to datetime if it's not already
if df['Application_month'].dtype != 'datetime64[ns]':
    df['Application_month'] = pd.to_datetime(df['Application_month'] + '-01')

# Separate train and test data
train_df = df[df['dataselection'] == 'Train']
test_df = df[df['dataselection'] == 'Test']

# Calculate decile bins for the entire training set
train_deciles = pd.qcut(train_df['s_apps_score'], 10, labels=False, retbins=True)
train_decile_bins = train_deciles[1]

# Function to calculate PSI using the pre-defined decile bins
def calculate_psi_with_bins(data_scores, decile_bins):
    """Calculates PSI using pre-defined decile bins."""
    data_deciles = pd.cut(data_scores, bins=decile_bins, labels=False, include_lowest=True)
    distribution = pd.Series(data_deciles).value_counts().sort_index() / len(data_scores)

    # Align with training distribution
    all_bins = range(10)  # Assuming 10 deciles
    distribution_aligned = distribution.reindex(all_bins, fill_value=0)
    train_dist_aligned = pd.Series(train_deciles[0]).value_counts().sort_index() / len(train_df['s_apps_score'])
    train_dist_aligned = train_dist_aligned.reindex(all_bins, fill_value=0)

    psi_values = (distribution_aligned - train_dist_aligned) * np.log(distribution_aligned / train_dist_aligned)
    return psi_values.sum()

# Calculate PSI for the entire training set
train_psi = calculate_psi_with_bins(train_df['s_apps_score'], train_decile_bins)

# Get the last month of the training set
last_train_month = train_df['Application_month'].max()
last_train_month_str = last_train_month.strftime('%Y-%m')

# Calculate monthly PSI for the test set
monthly_psi_results = []

# Add the train set PSI to the results (with the correct last month)
monthly_psi_results.append({
    'Month': last_train_month_str,  # Use the last month of the training set
    # 'loan_type': train_df['ln_loan_type'].iloc[0],  # Assuming loan_type is consistent in the train set
    'scorename': 's_apps_score',
    'DateCategory': 'Training',
    'psivalues': train_psi
})

# Calculate monthly PSI for the test set
for month in sorted(test_df['Application_month'].unique()):
    month_str = month.strftime('%Y-%m')
    month_df = test_df[test_df['Application_month'] == month]
    if not month_df.empty:
        month_psi = calculate_psi_with_bins(month_df['s_apps_score'], train_decile_bins)
        monthly_psi_results.append({
            'Month': month_str,
            # 'loan_type': month_df['ln_loan_type'].iloc[0],  # Assuming loan_type is consistent in the month
            'scorename': 's_apps_score',
            'DateCategory': 'Monthly',
            'psivalues': month_psi
        })

# Create the output DataFrame
s_apps_score_output_df = pd.DataFrame(monthly_psi_results)

Job ID ef1d7acf-b82f-4196-a5c9-224fa97c1cfc successfully executed: 100%|██████████|

d:\Anaconda3\envs\inc_env\lib\site-packages\google\cloud\bigquery\table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



Downloading: 100%|██████████|


In [36]:
s_apps_score_output_df.rename(columns={'psivalues':'s_apps_score_psivalues'}, inplace = True)

In [37]:
s_apps_score_output_df.head()

,Month,scorename,DateCategory,s_apps_score_psivalues
0,2024-06,s_apps_score,Training,0.000000
1,2024-07,s_apps_score,Monthly,0.034568
2,2024-08,s_apps_score,Monthly,0.035108
3,2024-09,s_apps_score,Monthly,0.045170
4,2024-10,s_apps_score,Monthly,0.044685


# sb_demo_score

In [38]:
sq = """
with base as 
(select 
  a.digitalLoanAccountId, 
  FORMAT_DATE('%Y-%m', a.ln_appln_submit_datetime) Application_month,
  FORMAT_DATE('%F', DATE_TRUNC(a.ln_appln_submit_datetime, WEEK(MONDAY))) as Appl_week_start_date,
  EXTRACT(WEEK(MONDAY) FROM a.ln_appln_submit_datetime) as Appl_week_number,
  a.ln_loan_type,
  case when date_trunc(a.ln_appln_submit_datetime, day) between '2023-07-01' and '2024-06-30' then 'Train'
       when date_trunc(a.ln_appln_submit_datetime, day) >= '2024-07-01' then 'Test'
       Else 'Other' end dataselection,
  a.sb_demo_score,
 from 
 prj-prod-dataplatform.risk_mart.sil_risk_ds_master_20230101_20250206 a
 where a.ln_loan_applied_flag = 1 and ln_dl_rule_reject_flag = 0
 and date_trunc(a.ln_appln_submit_datetime, day) >= '2023-04-01'
 )
 select * from base where sb_demo_score is not null and dataselection in ('Train', 'Test');"""
 
df = client.query(sq).to_dataframe(progress_bar_type='tqdm')
df.head()

Job ID 85dafb54-f6b6-4438-95f2-fd0d1f42dc83 successfully executed: 100%|██████████|

d:\Anaconda3\envs\inc_env\lib\site-packages\google\cloud\bigquery\table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



Downloading: 100%|██████████|


,digitalLoanAccountId,Application_month,Appl_week_start_date,Appl_week_number,ln_loan_type,dataselection,sb_demo_score
0,1f0d420f-746d-4f58-a4e7-50c345d701db,2023-10,2023-10-16,42,SIL-Instore,Train,0.111932
1,3632ebdd-470d-40e5-8de4-c1b807c981ab,2023-08,2023-08-07,32,SIL-Instore,Train,0.121969
2,b8d7faed-c257-4735-96bb-529eb556a1e9,2023-09,2023-09-18,38,SIL-Instore,Train,0.121990
3,065cf333-9d42-428d-92da-a7c8d18e57ea,2023-10,2023-10-09,41,SIL-Instore,Train,0.045061
4,adb459b3-9e99-41b0-a708-4914f4892188,2023-09,2023-09-18,38,SIL-Instore,Train,0.198594


In [39]:
import pandas as pd
import numpy as np
from datetime import datetime

# Replace this with your actual DataFrame loading process
sq = """
with base as 
(select 
  a.digitalLoanAccountId, 
  FORMAT_DATE('%Y-%m', a.ln_appln_submit_datetime) Application_month,
  FORMAT_DATE('%F', DATE_TRUNC(a.ln_appln_submit_datetime, WEEK(MONDAY))) as Appl_week_start_date,
  EXTRACT(WEEK(MONDAY) FROM a.ln_appln_submit_datetime) as Appl_week_number,
  a.ln_loan_type,
  case when date_trunc(a.ln_appln_submit_datetime, day) between '2023-07-01' and '2024-06-30' then 'Train'
       when date_trunc(a.ln_appln_submit_datetime, day) >= '2024-07-01' then 'Test'
       Else 'Other' end dataselection,
  a.sb_demo_score,
 from 
 prj-prod-dataplatform.risk_mart.sil_risk_ds_master_20230101_20250206 a
 where a.ln_loan_applied_flag = 1
 and date_trunc(a.ln_appln_submit_datetime, day) >= '2023-04-01'
 )
 select * from base where sb_demo_score is not null and dataselection in ('Train', 'Test');"""
df = client.query(sq).to_dataframe(progress_bar_type='tqdm')

# Convert Application_month to datetime if it's not already
if df['Application_month'].dtype != 'datetime64[ns]':
    df['Application_month'] = pd.to_datetime(df['Application_month'] + '-01')

# Separate train and test data
train_df = df[df['dataselection'] == 'Train']
test_df = df[df['dataselection'] == 'Test']

# Calculate decile bins for the entire training set
train_deciles = pd.qcut(train_df['sb_demo_score'], 10, labels=False, retbins=True)
train_decile_bins = train_deciles[1]
print(train_decile_bins)
# Function to calculate PSI using the pre-defined decile bins
def calculate_psi_with_bins(data_scores, decile_bins):
    """Calculates PSI using pre-defined decile bins."""
    data_deciles = pd.cut(data_scores, bins=decile_bins, labels=False, include_lowest=True)
    distribution = pd.Series(data_deciles).value_counts().sort_index() / len(data_scores)

    # Align with training distribution
    all_bins = range(10)  # Assuming 10 deciles
    distribution_aligned = distribution.reindex(all_bins, fill_value=0)
    train_dist_aligned = pd.Series(train_deciles[0]).value_counts().sort_index() / len(train_df['sb_demo_score'])
    train_dist_aligned = train_dist_aligned.reindex(all_bins, fill_value=0)

    psi_values = (distribution_aligned - train_dist_aligned) * np.log(distribution_aligned / train_dist_aligned)
    return psi_values.sum()

# Calculate PSI for the entire training set
train_psi = calculate_psi_with_bins(train_df['sb_demo_score'], train_decile_bins)

# Get the last month of the training set
last_train_month = train_df['Application_month'].max()
last_train_month_str = last_train_month.strftime('%Y-%m')

# Calculate monthly PSI for the test set
monthly_psi_results = []

# Add the train set PSI to the results (with the correct last month)
monthly_psi_results.append({
    'Month': last_train_month_str,  # Use the last month of the training set
    # 'loan_type': train_df['ln_loan_type'].iloc[0],  # Assuming loan_type is consistent in the train set
    'scorename': 'sb_demo_score',
    'DateCategory': 'Training',
    'psivalues': train_psi
})

# Calculate monthly PSI for the test set
for month in sorted(test_df['Application_month'].unique()):
    month_str = month.strftime('%Y-%m')
    month_df = test_df[test_df['Application_month'] == month]
    if not month_df.empty:
        month_psi = calculate_psi_with_bins(month_df['sb_demo_score'], train_decile_bins)
        monthly_psi_results.append({
            'Month': month_str,
            # 'loan_type': month_df['ln_loan_type'].iloc[0],  # Assuming loan_type is consistent in the month
            'scorename': 'sb_demo_score',
            'DateCategory': 'Monthly',
            'psivalues': month_psi
        })

# Create the output DataFrame
sb_demo_score_output_df = pd.DataFrame(monthly_psi_results)
sb_demo_score_output_df.rename(columns={'psivalues':'sb_demo_score_psivalues'}, inplace = True)
sb_demo_score_output_df

Job ID 9055daeb-b06b-4fc4-9b0f-90c92b79ea80 successfully executed: 100%|██████████|

d:\Anaconda3\envs\inc_env\lib\site-packages\google\cloud\bigquery\table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



Downloading: 100%|██████████|
[0.02599672 0.07539687 0.09369755 0.1089001  0.1232374  0.13752696
 0.1523714  0.16832625 0.18696915 0.21414669 0.49832412]


,Month,scorename,DateCategory,sb_demo_score_psivalues
0,2024-06,sb_demo_score,Training,0.000000
1,2024-07,sb_demo_score,Monthly,0.006962
2,2024-08,sb_demo_score,Monthly,0.007600
3,2024-09,sb_demo_score,Monthly,0.008773
4,2024-10,sb_demo_score,Monthly,0.009226
5,2024-11,sb_demo_score,Monthly,0.010188
6,2024-12,sb_demo_score,Monthly,0.024195
7,2025-01,sb_demo_score,Monthly,0.047343
8,2025-02,sb_demo_score,Monthly,0.046067


# s_cic_score

In [40]:
import pandas as pd
import numpy as np
from datetime import datetime

# Replace this with your actual DataFrame loading process
sq = """
with base as 
(select 
  a.digitalLoanAccountId, 
  FORMAT_DATE('%Y-%m', a.ln_appln_submit_datetime) Application_month,
  FORMAT_DATE('%F', DATE_TRUNC(a.ln_appln_submit_datetime, WEEK(MONDAY))) as Appl_week_start_date,
  EXTRACT(WEEK(MONDAY) FROM a.ln_appln_submit_datetime) as Appl_week_number,
  a.ln_loan_type,
  case when date_trunc(a.ln_appln_submit_datetime, day) between '2023-07-01' and '2024-06-30' then 'Train'
       when date_trunc(a.ln_appln_submit_datetime, day) >= '2024-07-01' then 'Test'
       Else 'Other' end dataselection,
  a.s_cic_score,
 from 
 prj-prod-dataplatform.risk_mart.sil_risk_ds_master_20230101_20250206 a
 where a.ln_loan_applied_flag = 1 and ln_dl_rule_reject_flag = 0
 and date_trunc(a.ln_appln_submit_datetime, day) >= '2023-04-01'
 )
 select * from base where s_cic_score is not null and dataselection in ('Train', 'Test');"""
df = client.query(sq).to_dataframe(progress_bar_type='tqdm')

# Convert Application_month to datetime if it's not already
if df['Application_month'].dtype != 'datetime64[ns]':
    df['Application_month'] = pd.to_datetime(df['Application_month'] + '-01')

# Separate train and test data
train_df = df[df['dataselection'] == 'Train']
test_df = df[df['dataselection'] == 'Test']

# Calculate decile bins for the entire training set
train_deciles = pd.qcut(train_df['s_cic_score'], 10, labels=False, retbins=True)
train_decile_bins = train_deciles[1]
print(train_decile_bins)
# Function to calculate PSI using the pre-defined decile bins
def calculate_psi_with_bins(data_scores, decile_bins):
    """Calculates PSI using pre-defined decile bins."""
    data_deciles = pd.cut(data_scores, bins=decile_bins, labels=False, include_lowest=True)
    distribution = pd.Series(data_deciles).value_counts().sort_index() / len(data_scores)

    # Align with training distribution
    all_bins = range(10)  # Assuming 10 deciles
    distribution_aligned = distribution.reindex(all_bins, fill_value=0)
    train_dist_aligned = pd.Series(train_deciles[0]).value_counts().sort_index() / len(train_df['s_cic_score'])
    train_dist_aligned = train_dist_aligned.reindex(all_bins, fill_value=0)

    psi_values = (distribution_aligned - train_dist_aligned) * np.log(distribution_aligned / train_dist_aligned)
    return psi_values.sum()

# Calculate PSI for the entire training set
train_psi = calculate_psi_with_bins(train_df['s_cic_score'], train_decile_bins)

# Get the last month of the training set
last_train_month = train_df['Application_month'].max()
last_train_month_str = last_train_month.strftime('%Y-%m')

# Calculate monthly PSI for the test set
monthly_psi_results = []

# Add the train set PSI to the results (with the correct last month)
monthly_psi_results.append({
    'Month': last_train_month_str,  # Use the last month of the training set
    # 'loan_type': train_df['ln_loan_type'].iloc[0],  # Assuming loan_type is consistent in the train set
    'scorename': 's_cic_score',
    'DateCategory': 'Training',
    'psivalues': train_psi
})

# Calculate monthly PSI for the test set
for month in sorted(test_df['Application_month'].unique()):
    month_str = month.strftime('%Y-%m')
    month_df = test_df[test_df['Application_month'] == month]
    if not month_df.empty:
        month_psi = calculate_psi_with_bins(month_df['s_cic_score'], train_decile_bins)
        monthly_psi_results.append({
            'Month': month_str,
            # 'loan_type': month_df['ln_loan_type'].iloc[0],  # Assuming loan_type is consistent in the month
            'scorename': 's_cic_score',
            'DateCategory': 'Monthly',
            'psivalues': month_psi
        })

# Create the output DataFrame
s_cic_score_output_df = pd.DataFrame(monthly_psi_results)
s_cic_score_output_df.rename(columns={'psivalues':'s_cic_score_psivalues'}, inplace = True)
s_cic_score_output_df

Job ID 361218fa-6ffe-49fa-9313-7ba465edc9a0 successfully executed: 100%|██████████|

d:\Anaconda3\envs\inc_env\lib\site-packages\google\cloud\bigquery\table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



Downloading: 100%|██████████|
[0.03142267 0.06219087 0.07031444 0.07868483 0.0876186  0.10219016
 0.12075118 0.14212345 0.15403213 0.18150904 0.63407415]


,Month,scorename,DateCategory,s_cic_score_psivalues
0,2024-06,s_cic_score,Training,0.000000
1,2024-07,s_cic_score,Monthly,0.050564
2,2024-08,s_cic_score,Monthly,0.036121
3,2024-09,s_cic_score,Monthly,0.024902
4,2024-10,s_cic_score,Monthly,0.016167
5,2024-11,s_cic_score,Monthly,0.011405
6,2024-12,s_cic_score,Monthly,0.031125
7,2025-01,s_cic_score,Monthly,0.041109
8,2025-02,s_cic_score,Monthly,0.040975


# sb_stack_score

In [41]:
import pandas as pd
import numpy as np
from datetime import datetime

# Assuming your DataFrame is called 'df' and has the structure from the image
# Replace this with your actual DataFrame loading process
sq = """
with base as 
(select 
  a.digitalLoanAccountId, 
  FORMAT_DATE('%Y-%m', a.ln_appln_submit_datetime) Application_month,
  FORMAT_DATE('%F', DATE_TRUNC(a.ln_appln_submit_datetime, WEEK(MONDAY))) as Appl_week_start_date,
  EXTRACT(WEEK(MONDAY) FROM a.ln_appln_submit_datetime) as Appl_week_number,
  a.ln_loan_type,
  case when date_trunc(a.ln_appln_submit_datetime, day) between '2023-07-01' and '2024-06-30' then 'Train'
       when date_trunc(a.ln_appln_submit_datetime, day) >= '2024-07-01' then 'Test'
       Else 'Other' end dataselection,
  a.sb_stack_score,
 from 
 prj-prod-dataplatform.risk_mart.sil_risk_ds_master_20230101_20250206 a
 where a.ln_loan_applied_flag = 1 and ln_dl_rule_reject_flag = 0
 and date_trunc(a.ln_appln_submit_datetime, day) >= '2023-04-01'
 )
 select * from base where sb_stack_score is not null and dataselection in ('Train', 'Test');"""
df = client.query(sq).to_dataframe(progress_bar_type='tqdm')

# Convert Application_month to datetime if it's not already
if df['Application_month'].dtype != 'datetime64[ns]':
    df['Application_month'] = pd.to_datetime(df['Application_month'] + '-01')

# Separate train and test data
train_df = df[df['dataselection'] == 'Train']
test_df = df[df['dataselection'] == 'Test']

# Calculate decile bins for the entire training set
train_deciles = pd.qcut(train_df['sb_stack_score'], 10, labels=False, retbins=True)
train_decile_bins = train_deciles[1]
print(train_decile_bins)
# Function to calculate PSI using the pre-defined decile bins
def calculate_psi_with_bins(data_scores, decile_bins):
    """Calculates PSI using pre-defined decile bins."""
    data_deciles = pd.cut(data_scores, bins=decile_bins, labels=False, include_lowest=True)
    distribution = pd.Series(data_deciles).value_counts().sort_index() / len(data_scores)

    # Align with training distribution
    all_bins = range(10)  # Assuming 10 deciles
    distribution_aligned = distribution.reindex(all_bins, fill_value=0)
    train_dist_aligned = pd.Series(train_deciles[0]).value_counts().sort_index() / len(train_df['sb_stack_score'])
    train_dist_aligned = train_dist_aligned.reindex(all_bins, fill_value=0)

    psi_values = (distribution_aligned - train_dist_aligned) * np.log(distribution_aligned / train_dist_aligned)
    return psi_values.sum()

# Calculate PSI for the entire training set
train_psi = calculate_psi_with_bins(train_df['sb_stack_score'], train_decile_bins)

# Get the last month of the training set
last_train_month = train_df['Application_month'].max()
last_train_month_str = last_train_month.strftime('%Y-%m')

# Calculate monthly PSI for the test set
monthly_psi_results = []

# Add the train set PSI to the results (with the correct last month)
monthly_psi_results.append({
    'Month': last_train_month_str,  # Use the last month of the training set
    # 'loan_type': train_df['ln_loan_type'].iloc[0],  # Assuming loan_type is consistent in the train set
    'scorename': 'sb_stack_score',
    'DateCategory': 'Training',
    'psivalues': train_psi
})

# Calculate monthly PSI for the test set
for month in sorted(test_df['Application_month'].unique()):
    month_str = month.strftime('%Y-%m')
    month_df = test_df[test_df['Application_month'] == month]
    if not month_df.empty:
        month_psi = calculate_psi_with_bins(month_df['sb_stack_score'], train_decile_bins)
        monthly_psi_results.append({
            'Month': month_str,
            # 'loan_type': month_df['ln_loan_type'].iloc[0],  # Assuming loan_type is consistent in the month
            'scorename': 'sb_stack_score',
            'DateCategory': 'Monthly',
            'psivalues': month_psi
        })

# Create the output DataFrame
sb_stack_score_output_df = pd.DataFrame(monthly_psi_results)
sb_stack_score_output_df.rename(columns={'psivalues':'sb_stack_score_psivalues'}, inplace = True)
sb_stack_score_output_df

Job ID e35d4038-ef64-4c9d-a6d6-1b77cdf3d67c successfully executed: 100%|██████████|

d:\Anaconda3\envs\inc_env\lib\site-packages\google\cloud\bigquery\table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



Downloading: 100%|██████████|
[0.00491896 0.02682666 0.03720064 0.04977516 0.0626489  0.07744465
 0.09431912 0.1141339  0.14068214 0.18507562 0.70542026]


,Month,scorename,DateCategory,sb_stack_score_psivalues
0,2024-06,sb_stack_score,Training,0.000000
1,2024-07,sb_stack_score,Monthly,0.035043
2,2024-08,sb_stack_score,Monthly,0.037148
3,2024-09,sb_stack_score,Monthly,0.047225
4,2024-10,sb_stack_score,Monthly,0.034103
5,2024-11,sb_stack_score,Monthly,0.034803
6,2024-12,sb_stack_score,Monthly,0.031321
7,2025-01,sb_stack_score,Monthly,0.175236
8,2025-02,sb_stack_score,Monthly,0.194751


# sa_stack_score

In [42]:
import pandas as pd
import numpy as np
from datetime import datetime

# Assuming your DataFrame is called 'df' and has the structure from the image
# Replace this with your actual DataFrame loading process
sq = """
with base as 
(select 
  a.digitalLoanAccountId, 
  FORMAT_DATE('%Y-%m', a.ln_appln_submit_datetime) Application_month,
  FORMAT_DATE('%F', DATE_TRUNC(a.ln_appln_submit_datetime, WEEK(MONDAY))) as Appl_week_start_date,
  EXTRACT(WEEK(MONDAY) FROM a.ln_appln_submit_datetime) as Appl_week_number,
  a.ln_loan_type,
  case when date_trunc(a.ln_appln_submit_datetime, day) between '2023-07-01' and '2024-06-30' then 'Train'
       when date_trunc(a.ln_appln_submit_datetime, day) >= '2024-07-01' then 'Test'
       Else 'Other' end dataselection,
  a.sa_stack_score,
 from 
 prj-prod-dataplatform.risk_mart.sil_risk_ds_master_20230101_20250206 a
 where a.ln_loan_applied_flag = 1  and ln_dl_rule_reject_flag = 0
 and date_trunc(a.ln_appln_submit_datetime, day) >= '2023-04-01'
 )
 select * from base where sa_stack_score is not null and dataselection in ('Train', 'Test');"""
df = client.query(sq).to_dataframe(progress_bar_type='tqdm')

# Convert Application_month to datetime if it's not already
if df['Application_month'].dtype != 'datetime64[ns]':
    df['Application_month'] = pd.to_datetime(df['Application_month'] + '-01')

# Separate train and test data
train_df = df[df['dataselection'] == 'Train']
test_df = df[df['dataselection'] == 'Test']

# Calculate decile bins for the entire training set
train_deciles = pd.qcut(train_df['sa_stack_score'], 10, labels=False, retbins=True)
train_decile_bins = train_deciles[1]
print(train_decile_bins)
# Function to calculate PSI using the pre-defined decile bins
def calculate_psi_with_bins(data_scores, decile_bins):
    """Calculates PSI using pre-defined decile bins."""
    data_deciles = pd.cut(data_scores, bins=decile_bins, labels=False, include_lowest=True)
    distribution = pd.Series(data_deciles).value_counts().sort_index() / len(data_scores)

    # Align with training distribution
    all_bins = range(10)  # Assuming 10 deciles
    distribution_aligned = distribution.reindex(all_bins, fill_value=0)
    train_dist_aligned = pd.Series(train_deciles[0]).value_counts().sort_index() / len(train_df['sa_stack_score'])
    train_dist_aligned = train_dist_aligned.reindex(all_bins, fill_value=0)

    psi_values = (distribution_aligned - train_dist_aligned) * np.log(distribution_aligned / train_dist_aligned)
    return psi_values.sum()

# Calculate PSI for the entire training set
train_psi = calculate_psi_with_bins(train_df['sa_stack_score'], train_decile_bins)

# Get the last month of the training set
last_train_month = train_df['Application_month'].max()
last_train_month_str = last_train_month.strftime('%Y-%m')

# Calculate monthly PSI for the test set
monthly_psi_results = []

# Add the train set PSI to the results (with the correct last month)
monthly_psi_results.append({
    'Month': last_train_month_str,  # Use the last month of the training set
    # 'loan_type': train_df['ln_loan_type'].iloc[0],  # Assuming loan_type is consistent in the train set
    'scorename': 'sa_stack_score',
    'DateCategory': 'Training',
    'psivalues': train_psi
})

# Calculate monthly PSI for the test set
for month in sorted(test_df['Application_month'].unique()):
    month_str = month.strftime('%Y-%m')
    month_df = test_df[test_df['Application_month'] == month]
    if not month_df.empty:
        month_psi = calculate_psi_with_bins(month_df['sa_stack_score'], train_decile_bins)
        monthly_psi_results.append({
            'Month': month_str,
            # 'loan_type': month_df['ln_loan_type'].iloc[0],  # Assuming loan_type is consistent in the month
            'scorename': 'sa_stack_score',
            'DateCategory': 'Monthly',
            'psivalues': month_psi
        })

# Create the output DataFrame
sa_stack_score_output_df = pd.DataFrame(monthly_psi_results)
sa_stack_score_output_df.rename(columns={'psivalues':'sa_stack_score_psivalues'}, inplace = True)
sa_stack_score_output_df


Job ID f11cbc48-371c-440e-b2fc-bb2fe3314ec8 successfully executed: 100%|██████████|

d:\Anaconda3\envs\inc_env\lib\site-packages\google\cloud\bigquery\table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



Downloading: 100%|██████████|
[0.00588077 0.02600035 0.03525914 0.04530016 0.05783029 0.07148977
 0.08640558 0.10476441 0.12909164 0.17078101 0.79084583]


,Month,scorename,DateCategory,sa_stack_score_psivalues
0,2024-06,sa_stack_score,Training,0.000000
1,2024-07,sa_stack_score,Monthly,0.051323
2,2024-08,sa_stack_score,Monthly,0.051244
3,2024-09,sa_stack_score,Monthly,0.058801
4,2024-10,sa_stack_score,Monthly,0.045726
5,2024-11,sa_stack_score,Monthly,0.050612
6,2024-12,sa_stack_score,Monthly,0.036002
7,2025-01,sa_stack_score,Monthly,0.054459
8,2025-02,sa_stack_score,Monthly,0.059482


# c_credo_score_output_df

In [43]:
import pandas as pd
import numpy as np
from datetime import datetime

# Assuming your DataFrame is called 'df' and has the structure from the image
# Replace this with your actual DataFrame loading process
sq = """
with base as 
(select 
  a.digitalLoanAccountId, 
  FORMAT_DATE('%Y-%m', a.ln_appln_submit_datetime) Application_month,
  FORMAT_DATE('%F', DATE_TRUNC(a.ln_appln_submit_datetime, WEEK(MONDAY))) as Appl_week_start_date,
  EXTRACT(WEEK(MONDAY) FROM a.ln_appln_submit_datetime) as Appl_week_number,
  a.ln_loan_type,
  case when date_trunc(a.ln_appln_submit_datetime, day) between '2023-07-01' and '2024-06-30' then 'Train'
       when date_trunc(a.ln_appln_submit_datetime, day) >= '2024-07-01' then 'Test'
       Else 'Other' end dataselection,
  a.c_credo_score,
 from 
 prj-prod-dataplatform.risk_mart.sil_risk_ds_master_20230101_20250206 a
 where a.ln_loan_applied_flag = 1  and ln_dl_rule_reject_flag = 0
 and date_trunc(a.ln_appln_submit_datetime, day) >= '2023-04-01'
 )
 select * from base where c_credo_score is not null and dataselection in ('Train', 'Test');"""
df = client.query(sq).to_dataframe(progress_bar_type='tqdm')

# Convert Application_month to datetime if it's not already
if df['Application_month'].dtype != 'datetime64[ns]':
    df['Application_month'] = pd.to_datetime(df['Application_month'] + '-01')

# Separate train and test data
train_df = df[df['dataselection'] == 'Train']
test_df = df[df['dataselection'] == 'Test']

# Calculate decile bins for the entire training set
train_deciles = pd.qcut(train_df['c_credo_score'], 10, labels=False, retbins=True)
train_decile_bins = train_deciles[1]
print(train_decile_bins)
# Function to calculate PSI using the pre-defined decile bins
def calculate_psi_with_bins(data_scores, decile_bins):
    """Calculates PSI using pre-defined decile bins."""
    data_deciles = pd.cut(data_scores, bins=decile_bins, labels=False, include_lowest=True)
    distribution = pd.Series(data_deciles).value_counts().sort_index() / len(data_scores)

    # Align with training distribution
    all_bins = range(10)  # Assuming 10 deciles
    distribution_aligned = distribution.reindex(all_bins, fill_value=0)
    train_dist_aligned = pd.Series(train_deciles[0]).value_counts().sort_index() / len(train_df['c_credo_score'])
    train_dist_aligned = train_dist_aligned.reindex(all_bins, fill_value=0)

    psi_values = (distribution_aligned - train_dist_aligned) * np.log(distribution_aligned / train_dist_aligned)
    return psi_values.sum()

# Calculate PSI for the entire training set
train_psi = calculate_psi_with_bins(train_df['c_credo_score'], train_decile_bins)

# Get the last month of the training set
last_train_month = train_df['Application_month'].max()
last_train_month_str = last_train_month.strftime('%Y-%m')

# Calculate monthly PSI for the test set
monthly_psi_results = []

# Add the train set PSI to the results (with the correct last month)
monthly_psi_results.append({
    'Month': last_train_month_str,  # Use the last month of the training set
    # 'loan_type': train_df['ln_loan_type'].iloc[0],  # Assuming loan_type is consistent in the train set
    'scorename': 'c_credo_score',
    'DateCategory': 'Training',
    'psivalues': train_psi
})

# Calculate monthly PSI for the test set
for month in sorted(test_df['Application_month'].unique()):
    month_str = month.strftime('%Y-%m')
    month_df = test_df[test_df['Application_month'] == month]
    if not month_df.empty:
        month_psi = calculate_psi_with_bins(month_df['c_credo_score'], train_decile_bins)
        monthly_psi_results.append({
            'Month': month_str,
            # 'loan_type': month_df['ln_loan_type'].iloc[0],  # Assuming loan_type is consistent in the month
            'scorename': 'c_credo_score',
            'DateCategory': 'Monthly',
            'psivalues': month_psi
        })

# Create the output DataFrame
c_credo_score_output_df = pd.DataFrame(monthly_psi_results)
c_credo_score_output_df.rename(columns={'psivalues':'c_credo_score_psivalues'}, inplace = True)

Job ID 187223bf-7d1c-4436-a365-a20b694df3b9 successfully executed: 100%|██████████|

d:\Anaconda3\envs\inc_env\lib\site-packages\google\cloud\bigquery\table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



Downloading: 100%|██████████|
[0.01944472 0.15469384 0.1915581  0.22351158 0.25562673 0.28830582
 0.3234565  0.363668   0.41261439 0.48274307 0.85616948]


In [44]:
c_credo_score_output_df

,Month,scorename,DateCategory,c_credo_score_psivalues
0,2024-06,c_credo_score,Training,0.000000
1,2024-07,c_credo_score,Monthly,0.007487
2,2024-08,c_credo_score,Monthly,0.004699
3,2024-09,c_credo_score,Monthly,0.004640
4,2024-10,c_credo_score,Monthly,0.002253
5,2024-11,c_credo_score,Monthly,0.001974
6,2024-12,c_credo_score,Monthly,0.001777
7,2025-01,c_credo_score,Monthly,0.026723
8,2025-02,c_credo_score,Monthly,0.065453


# s_credo_score

In [45]:
import pandas as pd
import numpy as np
from datetime import datetime

# Assuming your DataFrame is called 'df' and has the structure from the image
# Replace this with your actual DataFrame loading process
sq = """
with base as 
(select 
  a.digitalLoanAccountId, 
  FORMAT_DATE('%Y-%m', a.ln_appln_submit_datetime) Application_month,
  FORMAT_DATE('%F', DATE_TRUNC(a.ln_appln_submit_datetime, WEEK(MONDAY))) as Appl_week_start_date,
  EXTRACT(WEEK(MONDAY) FROM a.ln_appln_submit_datetime) as Appl_week_number,
  a.ln_loan_type,
  case when date_trunc(a.ln_appln_submit_datetime, day) between '2023-07-01' and '2024-06-30' then 'Train'
       when date_trunc(a.ln_appln_submit_datetime, day) >= '2024-07-01' then 'Test'
       Else 'Other' end dataselection,
  a.s_credo_score,
 from 
 prj-prod-dataplatform.risk_mart.sil_risk_ds_master_20230101_20250206 a
 where a.ln_loan_applied_flag = 1  and ln_dl_rule_reject_flag = 0
 and date_trunc(a.ln_appln_submit_datetime, day) >= '2023-04-01'
 )
 select * from base where s_credo_score is not null and dataselection in ('Train', 'Test');"""
df = client.query(sq).to_dataframe(progress_bar_type='tqdm')

# Convert Application_month to datetime if it's not already
if df['Application_month'].dtype != 'datetime64[ns]':
    df['Application_month'] = pd.to_datetime(df['Application_month'] + '-01')

# Separate train and test data
train_df = df[df['dataselection'] == 'Train']
test_df = df[df['dataselection'] == 'Test']

# Calculate decile bins for the entire training set
train_deciles = pd.qcut(train_df['s_credo_score'], 10, labels=False, retbins=True)
train_decile_bins = train_deciles[1]
print(train_decile_bins)
# Function to calculate PSI using the pre-defined decile bins
def calculate_psi_with_bins(data_scores, decile_bins):
    """Calculates PSI using pre-defined decile bins."""
    data_deciles = pd.cut(data_scores, bins=decile_bins, labels=False, include_lowest=True)
    distribution = pd.Series(data_deciles).value_counts().sort_index() / len(data_scores)

    # Align with training distribution
    all_bins = range(10)  # Assuming 10 deciles
    distribution_aligned = distribution.reindex(all_bins, fill_value=0)
    train_dist_aligned = pd.Series(train_deciles[0]).value_counts().sort_index() / len(train_df['s_credo_score'])
    train_dist_aligned = train_dist_aligned.reindex(all_bins, fill_value=0)

    psi_values = (distribution_aligned - train_dist_aligned) * np.log(distribution_aligned / train_dist_aligned)
    return psi_values.sum()

# Calculate PSI for the entire training set
train_psi = calculate_psi_with_bins(train_df['s_credo_score'], train_decile_bins)

# Get the last month of the training set
last_train_month = train_df['Application_month'].max()
last_train_month_str = last_train_month.strftime('%Y-%m')

# Calculate monthly PSI for the test set
monthly_psi_results = []

# Add the train set PSI to the results (with the correct last month)
monthly_psi_results.append({
    'Month': last_train_month_str,  # Use the last month of the training set
    # 'loan_type': train_df['ln_loan_type'].iloc[0],  # Assuming loan_type is consistent in the train set
    'scorename': 's_credo_score',
    'DateCategory': 'Training',
    'psivalues': train_psi
})

# Calculate monthly PSI for the test set
for month in sorted(test_df['Application_month'].unique()):
    month_str = month.strftime('%Y-%m')
    month_df = test_df[test_df['Application_month'] == month]
    if not month_df.empty:
        month_psi = calculate_psi_with_bins(month_df['s_credo_score'], train_decile_bins)
        monthly_psi_results.append({
            'Month': month_str,
            # 'loan_type': month_df['ln_loan_type'].iloc[0],  # Assuming loan_type is consistent in the month
            'scorename': 's_credo_score',
            'DateCategory': 'Monthly',
            'psivalues': month_psi
        })

# Create the output DataFrame
s_credo_score_output_df = pd.DataFrame(monthly_psi_results)
s_credo_score_output_df.rename(columns= {'psivalues':'s_credo_score_psivalues'}, inplace = True)
s_credo_score_output_df

Job ID e80ac3a3-9d29-420f-aaf3-69f7305541fc successfully executed: 100%|██████████|

d:\Anaconda3\envs\inc_env\lib\site-packages\google\cloud\bigquery\table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



Downloading: 100%|██████████|
[0.01777745 0.05784676 0.07000222 0.08029911 0.09058302 0.10111116
 0.11278586 0.12735223 0.14680204 0.17729976 0.44602214]


,Month,scorename,DateCategory,s_credo_score_psivalues
0,2024-06,s_credo_score,Training,0.000000
1,2024-07,s_credo_score,Monthly,0.007786
2,2024-08,s_credo_score,Monthly,0.003202
3,2024-09,s_credo_score,Monthly,0.009541
4,2024-10,s_credo_score,Monthly,0.014234
5,2024-11,s_credo_score,Monthly,0.005961
6,2024-12,s_credo_score,Monthly,0.004833
7,2025-01,s_credo_score,Monthly,0.004068
8,2025-02,s_credo_score,Monthly,0.010102


# fu_credo_score

In [46]:
import pandas as pd
import numpy as np
from datetime import datetime

# Assuming your DataFrame is called 'df' and has the structure from the image
# Replace this with your actual DataFrame loading process
sq = """
with base as 
(select 
  a.digitalLoanAccountId, 
  FORMAT_DATE('%Y-%m', a.ln_appln_submit_datetime) Application_month,
  FORMAT_DATE('%F', DATE_TRUNC(a.ln_appln_submit_datetime, WEEK(MONDAY))) as Appl_week_start_date,
  EXTRACT(WEEK(MONDAY) FROM a.ln_appln_submit_datetime) as Appl_week_number,
  a.ln_loan_type,
  case when date_trunc(a.ln_appln_submit_datetime, day) between '2023-07-01' and '2024-06-30' then 'Train'
       when date_trunc(a.ln_appln_submit_datetime, day) >= '2024-07-01' then 'Test'
       Else 'Other' end dataselection,
  a.fu_credo_score,
 from 
 prj-prod-dataplatform.risk_mart.sil_risk_ds_master_20230101_20250206 a
 where a.ln_loan_applied_flag = 1  and ln_dl_rule_reject_flag = 0
 and date_trunc(a.ln_appln_submit_datetime, day) >= '2023-04-01'
 )
 select * from base where fu_credo_score is not null and dataselection in ('Train', 'Test');"""
df = client.query(sq).to_dataframe(progress_bar_type='tqdm')

# Convert Application_month to datetime if it's not already
if df['Application_month'].dtype != 'datetime64[ns]':
    df['Application_month'] = pd.to_datetime(df['Application_month'] + '-01')

# Separate train and test data
train_df = df[df['dataselection'] == 'Train']
test_df = df[df['dataselection'] == 'Test']

# Calculate decile bins for the entire training set
train_deciles = pd.qcut(train_df['fu_credo_score'], 10, labels=False, retbins=True)
train_decile_bins = train_deciles[1]
print(train_decile_bins)
# Function to calculate PSI using the pre-defined decile bins
def calculate_psi_with_bins(data_scores, decile_bins):
    """Calculates PSI using pre-defined decile bins."""
    data_deciles = pd.cut(data_scores, bins=decile_bins, labels=False, include_lowest=True)
    distribution = pd.Series(data_deciles).value_counts().sort_index() / len(data_scores)

    # Align with training distribution
    all_bins = range(10)  # Assuming 10 deciles
    distribution_aligned = distribution.reindex(all_bins, fill_value=0)
    train_dist_aligned = pd.Series(train_deciles[0]).value_counts().sort_index() / len(train_df['fu_credo_score'])
    train_dist_aligned = train_dist_aligned.reindex(all_bins, fill_value=0)

    psi_values = (distribution_aligned - train_dist_aligned) * np.log(distribution_aligned / train_dist_aligned)
    return psi_values.sum()

# Calculate PSI for the entire training set
train_psi = calculate_psi_with_bins(train_df['fu_credo_score'], train_decile_bins)

# Get the last month of the training set
last_train_month = train_df['Application_month'].max()
last_train_month_str = last_train_month.strftime('%Y-%m')

# Calculate monthly PSI for the test set
monthly_psi_results = []

# Add the train set PSI to the results (with the correct last month)
monthly_psi_results.append({
    'Month': last_train_month_str,  # Use the last month of the training set
    # 'loan_type': train_df['ln_loan_type'].iloc[0],  # Assuming loan_type is consistent in the train set
    'scorename': 'fu_credo_score',
    'DateCategory': 'Training',
    'psivalues': train_psi
})

# Calculate monthly PSI for the test set
for month in sorted(test_df['Application_month'].unique()):
    month_str = month.strftime('%Y-%m')
    month_df = test_df[test_df['Application_month'] == month]
    if not month_df.empty:
        month_psi = calculate_psi_with_bins(month_df['fu_credo_score'], train_decile_bins)
        monthly_psi_results.append({
            'Month': month_str,
            # 'loan_type': month_df['ln_loan_type'].iloc[0],  # Assuming loan_type is consistent in the month
            'scorename': 'fu_credo_score',
            'DateCategory': 'Monthly',
            'psivalues': month_psi
        })

# Create the output DataFrame
fu_credo_score_output_df = pd.DataFrame(monthly_psi_results)
fu_credo_score_output_df.rename(columns={'psivalues':'fu_credo_score_psivalues'}, inplace = True)
fu_credo_score_output_df

Job ID a47e0b2d-cfa8-4d5f-b340-d83ec3b14c08 successfully executed: 100%|██████████|

d:\Anaconda3\envs\inc_env\lib\site-packages\google\cloud\bigquery\table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



Downloading: 100%|██████████|
[0.00198555 0.02799237 0.04116532 0.05471296 0.06982464 0.08789226
 0.11049724 0.13912719 0.1786316  0.23505313 0.86342827]


,Month,scorename,DateCategory,fu_credo_score_psivalues
0,2024-06,fu_credo_score,Training,0.000000
1,2024-07,fu_credo_score,Monthly,0.004743
2,2024-08,fu_credo_score,Monthly,0.005782
3,2024-09,fu_credo_score,Monthly,0.006832
4,2024-10,fu_credo_score,Monthly,0.015086
5,2024-11,fu_credo_score,Monthly,0.010949
6,2024-12,fu_credo_score,Monthly,0.010884
7,2025-01,fu_credo_score,Monthly,0.011854
8,2025-02,fu_credo_score,Monthly,0.024306


# r_credo_score

In [47]:
import pandas as pd
import numpy as np
from datetime import datetime

# Assuming your DataFrame is called 'df' and has the structure from the image
# Replace this with your actual DataFrame loading process
sq = """
with base as 
(select 
  a.digitalLoanAccountId, 
  FORMAT_DATE('%Y-%m', a.ln_appln_submit_datetime) Application_month,
  FORMAT_DATE('%F', DATE_TRUNC(a.ln_appln_submit_datetime, WEEK(MONDAY))) as Appl_week_start_date,
  EXTRACT(WEEK(MONDAY) FROM a.ln_appln_submit_datetime) as Appl_week_number,
  a.ln_loan_type,
  case when date_trunc(a.ln_appln_submit_datetime, day) between '2023-07-01' and '2024-06-30' then 'Train'
       when date_trunc(a.ln_appln_submit_datetime, day) >= '2024-07-01' then 'Test'
       Else 'Other' end dataselection,
  a.r_credo_score,
 from 
 prj-prod-dataplatform.risk_mart.sil_risk_ds_master_20230101_20250206 a
 where a.ln_loan_applied_flag = 1  and ln_dl_rule_reject_flag = 0
 and date_trunc(a.ln_appln_submit_datetime, day) >= '2023-04-01'
 )
 select * from base where r_credo_score is not null and dataselection in ('Train', 'Test');"""
df = client.query(sq).to_dataframe(progress_bar_type='tqdm')

# Convert Application_month to datetime if it's not already
if df['Application_month'].dtype != 'datetime64[ns]':
    df['Application_month'] = pd.to_datetime(df['Application_month'] + '-01')

# Separate train and test data
train_df = df[df['dataselection'] == 'Train']
test_df = df[df['dataselection'] == 'Test']

# Calculate decile bins for the entire training set
train_deciles = pd.qcut(train_df['r_credo_score'], 10, labels=False, retbins=True)
train_decile_bins = train_deciles[1]
print(train_decile_bins)
# Function to calculate PSI using the pre-defined decile bins
def calculate_psi_with_bins(data_scores, decile_bins):
    """Calculates PSI using pre-defined decile bins."""
    data_deciles = pd.cut(data_scores, bins=decile_bins, labels=False, include_lowest=True)
    distribution = pd.Series(data_deciles).value_counts().sort_index() / len(data_scores)

    # Align with training distribution
    all_bins = range(10)  # Assuming 10 deciles
    distribution_aligned = distribution.reindex(all_bins, fill_value=0)
    train_dist_aligned = pd.Series(train_deciles[0]).value_counts().sort_index() / len(train_df['r_credo_score'])
    train_dist_aligned = train_dist_aligned.reindex(all_bins, fill_value=0)

    psi_values = (distribution_aligned - train_dist_aligned) * np.log(distribution_aligned / train_dist_aligned)
    return psi_values.sum()

# Calculate PSI for the entire training set
train_psi = calculate_psi_with_bins(train_df['r_credo_score'], train_decile_bins)

# Get the last month of the training set
last_train_month = train_df['Application_month'].max()
last_train_month_str = last_train_month.strftime('%Y-%m')

# Calculate monthly PSI for the test set
monthly_psi_results = []

# Add the train set PSI to the results (with the correct last month)
monthly_psi_results.append({
    'Month': last_train_month_str,  # Use the last month of the training set
    # 'loan_type': train_df['ln_loan_type'].iloc[0],  # Assuming loan_type is consistent in the train set
    'scorename': 'r_credo_score',
    'DateCategory': 'Training',
    'psivalues': train_psi
})

# Calculate monthly PSI for the test set
for month in sorted(test_df['Application_month'].unique()):
    month_str = month.strftime('%Y-%m')
    month_df = test_df[test_df['Application_month'] == month]
    if not month_df.empty:
        month_psi = calculate_psi_with_bins(month_df['r_credo_score'], train_decile_bins)
        monthly_psi_results.append({
            'Month': month_str,
            # 'loan_type': month_df['ln_loan_type'].iloc[0],  # Assuming loan_type is consistent in the month
            'scorename': 'r_credo_score',
            'DateCategory': 'Monthly',
            'psivalues': month_psi
        })

# Create the output DataFrame
r_credo_score_output_df = pd.DataFrame(monthly_psi_results)
r_credo_score_output_df.rename(columns={'psivalues':'r_credo_score_psivalues'}, inplace = True)
r_credo_score_output_df


Job ID 58201152-c9c8-42fc-b0bd-bc67a9dbe12f successfully executed: 100%|██████████|

d:\Anaconda3\envs\inc_env\lib\site-packages\google\cloud\bigquery\table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



Downloading: 100%|██████████|
[0.00715845 0.09963035 0.13008679 0.16020453 0.19165548 0.22515524
 0.26229733 0.30518285 0.35905343 0.43906661 0.87489346]


,Month,scorename,DateCategory,r_credo_score_psivalues
0,2024-06,r_credo_score,Training,0.000000
1,2024-07,r_credo_score,Monthly,0.006297
2,2024-08,r_credo_score,Monthly,0.004650
3,2024-09,r_credo_score,Monthly,0.004957
4,2024-10,r_credo_score,Monthly,0.005452
5,2024-11,r_credo_score,Monthly,0.002148
6,2024-12,r_credo_score,Monthly,0.002151
7,2025-01,r_credo_score,Monthly,0.057639
8,2025-02,r_credo_score,Monthly,0.050379


# gen_credo_score

In [48]:
import pandas as pd
import numpy as np
from datetime import datetime

# Assuming your DataFrame is called 'df' and has the structure from the image
# Replace this with your actual DataFrame loading process
sq = """
with base as 
(select 
  a.digitalLoanAccountId, 
  FORMAT_DATE('%Y-%m', a.ln_appln_submit_datetime) Application_month,
  FORMAT_DATE('%F', DATE_TRUNC(a.ln_appln_submit_datetime, WEEK(MONDAY))) as Appl_week_start_date,
  EXTRACT(WEEK(MONDAY) FROM a.ln_appln_submit_datetime) as Appl_week_number,
  a.ln_loan_type,
  case when date_trunc(a.ln_appln_submit_datetime, day) between '2023-07-01' and '2024-06-30' then 'Train'
       when date_trunc(a.ln_appln_submit_datetime, day) >= '2024-07-01' then 'Test'
       Else 'Other' end dataselection,
  a.gen_credo_score,
 from 
 prj-prod-dataplatform.risk_mart.sil_risk_ds_master_20230101_20250206 a
 where a.ln_loan_applied_flag = 1  and ln_dl_rule_reject_flag = 0
 and date_trunc(a.ln_appln_submit_datetime, day) >= '2023-04-01'
 )
 select * from base where gen_credo_score is not null and dataselection in ('Train', 'Test');"""
df = client.query(sq).to_dataframe(progress_bar_type='tqdm')

# Convert Application_month to datetime if it's not already
if df['Application_month'].dtype != 'datetime64[ns]':
    df['Application_month'] = pd.to_datetime(df['Application_month'] + '-01')

# Separate train and test data
train_df = df[df['dataselection'] == 'Train']
test_df = df[df['dataselection'] == 'Test']

# Calculate decile bins for the entire training set
train_deciles = pd.qcut(train_df['gen_credo_score'], 10, labels=False, retbins=True)
train_decile_bins = train_deciles[1]
print(train_decile_bins)
# Function to calculate PSI using the pre-defined decile bins
def calculate_psi_with_bins(data_scores, decile_bins):
    """Calculates PSI using pre-defined decile bins."""
    data_deciles = pd.cut(data_scores, bins=decile_bins, labels=False, include_lowest=True)
    distribution = pd.Series(data_deciles).value_counts().sort_index() / len(data_scores)

    # Align with training distribution
    all_bins = range(10)  # Assuming 10 deciles
    distribution_aligned = distribution.reindex(all_bins, fill_value=0)
    train_dist_aligned = pd.Series(train_deciles[0]).value_counts().sort_index() / len(train_df['gen_credo_score'])
    train_dist_aligned = train_dist_aligned.reindex(all_bins, fill_value=0)

    psi_values = (distribution_aligned - train_dist_aligned) * np.log(distribution_aligned / train_dist_aligned)
    return psi_values.sum()

# Calculate PSI for the entire training set
train_psi = calculate_psi_with_bins(train_df['gen_credo_score'], train_decile_bins)

# Get the last month of the training set
last_train_month = train_df['Application_month'].max()
last_train_month_str = last_train_month.strftime('%Y-%m')

# Calculate monthly PSI for the test set
monthly_psi_results = []

# Add the train set PSI to the results (with the correct last month)
monthly_psi_results.append({
    'Month': last_train_month_str,  # Use the last month of the training set
    # 'loan_type': train_df['ln_loan_type'].iloc[0],  # Assuming loan_type is consistent in the train set
    'scorename': 'gen_credo_score',
    'DateCategory': 'Training',
    'psivalues': train_psi
})

# Calculate monthly PSI for the test set
for month in sorted(test_df['Application_month'].unique()):
    month_str = month.strftime('%Y-%m')
    month_df = test_df[test_df['Application_month'] == month]
    if not month_df.empty:
        month_psi = calculate_psi_with_bins(month_df['gen_credo_score'], train_decile_bins)
        monthly_psi_results.append({
            'Month': month_str,
            # 'loan_type': month_df['ln_loan_type'].iloc[0],  # Assuming loan_type is consistent in the month
            'scorename': 'gen_credo_score',
            'DateCategory': 'Monthly',
            'psivalues': month_psi
        })

# Create the output DataFrame
gen_credo_score_output_df = pd.DataFrame(monthly_psi_results)
gen_credo_score_output_df.rename(columns={'psivalues':'gen_credo_score_psivalues'}, inplace = True)
gen_credo_score_output_df

Job ID f131d0b0-1c8d-4a18-98ff-be3f4dc1e4d8 successfully executed: 100%|██████████|

d:\Anaconda3\envs\inc_env\lib\site-packages\google\cloud\bigquery\table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



Downloading: 100%|██████████|
[0.01431733 0.05366266 0.06762727 0.08016847 0.09249734 0.10529823
 0.11971361 0.13694468 0.15963359 0.1960659  0.57325398]


,Month,scorename,DateCategory,gen_credo_score_psivalues
0,2024-06,gen_credo_score,Training,0.000000
1,2024-07,gen_credo_score,Monthly,0.015349
2,2024-08,gen_credo_score,Monthly,0.009628
3,2024-09,gen_credo_score,Monthly,0.013419
4,2024-10,gen_credo_score,Monthly,0.020813
5,2024-11,gen_credo_score,Monthly,0.013389
6,2024-12,gen_credo_score,Monthly,0.013169
7,2025-01,gen_credo_score,Monthly,0.014192
8,2025-02,gen_credo_score,Monthly,0.009824


# Combining dataframes

In [49]:
# import functools

# dataframes = [s_apps_score_output_df, sb_demo_score_output_df, s_cic_score_output_df, sb_stack_score_output_df,sa_stack_score_output_df, c_credo_score_output_df, s_credo_score_output_df, fu_credo_score_output_df,
#               r_credo_score_output_df, gen_credo_score_output_df]
# common_columns = ['Month', 'scorename', 'DateCategory', 'psivalues']

# def merge_dataframes(df1, df2):
#     return pd.merge(df1, df2, on=common_columns, how='outer')

# final_df = functools.reduce(merge_dataframes, dataframes)

# final_df.columns.values

In [50]:
import pandas as pd

def concatenate_dataframes(dataframe_list):
    """
    Concatenates a list of Pandas DataFrames into a single DataFrame.

    Args:
        dataframe_list: A list of Pandas DataFrames to concatenate.

    Returns:
        A single concatenated Pandas DataFrame, or None if the input list is empty.
    """
    if not dataframe_list:
        return None  # Return None if the list is empty

    try:
        concatenated_df = pd.concat(dataframe_list, ignore_index=True)
        return concatenated_df
    except Exception as e:
        print(f"An error occurred during concatenation: {e}")
        return None

# Example usage (assuming your DataFrames are already defined):

# Replace these with your actual DataFrames
# s_apps_score_output_df = ...
# sb_demo_score_output_df = ...
# s_cic_score_output_df = ...
# sb_stack_score_output_df = ...
# sa_stack_score_output_df = ...
# c_credo_score_output_df = ...
# s_credo_score_output_df = ...
# fu_credo_score_output_df = ...
# r_credo_score_output_df = ...
# gen_credo_score_output_df = ...

dataframe_list = [
    s_apps_score_output_df,
    sb_demo_score_output_df,
    s_cic_score_output_df,
    sb_stack_score_output_df,
    sa_stack_score_output_df,
    c_credo_score_output_df,
    s_credo_score_output_df,
    fu_credo_score_output_df,
    r_credo_score_output_df,
    gen_credo_score_output_df,
]

concatenated_result = concatenate_dataframes(dataframe_list)

if concatenated_result is not None:
    print(concatenated_result)
else:
    print("Concatenation failed or the input list was empty.")

      Month        scorename DateCategory  s_apps_score_psivalues  \
0   2024-06     s_apps_score     Training                0.000000   
1   2024-07     s_apps_score      Monthly                0.034568   
2   2024-08     s_apps_score      Monthly                0.035108   
3   2024-09     s_apps_score      Monthly                0.045170   
4   2024-10     s_apps_score      Monthly                0.044685   
..      ...              ...          ...                     ...   
85  2024-10  gen_credo_score      Monthly                     NaN   
86  2024-11  gen_credo_score      Monthly                     NaN   
87  2024-12  gen_credo_score      Monthly                     NaN   
88  2025-01  gen_credo_score      Monthly                     NaN   
89  2025-02  gen_credo_score      Monthly                     NaN   

    sb_demo_score_psivalues  s_cic_score_psivalues  sb_stack_score_psivalues  \
0                       NaN                    NaN                       NaN   
1          

In [51]:
concatenated_result.dtypes

Month                         object
scorename                     object
DateCategory                  object
s_apps_score_psivalues       float64
sb_demo_score_psivalues      float64
s_cic_score_psivalues        float64
sb_stack_score_psivalues     float64
sa_stack_score_psivalues     float64
c_credo_score_psivalues      float64
s_credo_score_psivalues      float64
fu_credo_score_psivalues     float64
r_credo_score_psivalues      float64
gen_credo_score_psivalues    float64
dtype: object

In [52]:
sq = """drop table if exists prj-prod-dataplatform.dap_ds_poweruser_playground.Model_Psi;"""

client.query(sq)

QueryJob<project=prj-prod-dataplatform, location=asia-southeast1, id=bca85b6d-a603-4c86-a384-39df974839aa>

In [53]:
import pandas as pd
from google.cloud import bigquery

# Create a BigQuery client
client = bigquery.Client('prj-prod-dataplatform')

# Define your table schema
table_schema = [
    bigquery.SchemaField('Month', 'STRING'),
    bigquery.SchemaField('scorename', 'STRING'),
    bigquery.SchemaField('DateCategory', 'STRING'),
    bigquery.SchemaField('s_apps_score_psivalues', 'FLOAT64'),
    bigquery.SchemaField('sb_demo_score_psivalues', 'FLOAT64'),
    bigquery.SchemaField('s_cic_score_psivalues', 'FLOAT64'),
    bigquery.SchemaField('sb_stack_score_psivalues', 'FLOAT64'),
    bigquery.SchemaField('sa_stack_score_psivalues', 'FLOAT64'),
    bigquery.SchemaField('c_credo_score_psivalues', 'FLOAT64'),
    bigquery.SchemaField('s_credo_score_psivalue', 'FLOAT64'),
    bigquery.SchemaField('fu_credo_score_psivalues', 'FLOAT64'),
    bigquery.SchemaField('r_credo_score_psivalues', 'FLOAT64'),
    bigquery.SchemaField('gen_credo_score_psivalues', 'FLOAT64'),
   
]

# Create your BigQuery table
table_id = 'prj-prod-dataplatform.dap_ds_poweruser_playground.Model_Psi'
table = bigquery.Table(table_id, schema=table_schema)
table = client.create_table(table)

# Load your DataFrame into BigQuery
job_config = bigquery.LoadJobConfig(
    write_disposition='WRITE_TRUNCATE'
)

load_job = client.load_table_from_dataframe(
    concatenated_result, table_id, job_config=job_config
)

load_job.result()

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=ce6939bc-ef57-4a45-bc6c-7976374138f0>

In [54]:
concatenated_result.head()

,Month,scorename,DateCategory,s_apps_score_psivalues,sb_demo_score_psivalues,s_cic_score_psivalues,sb_stack_score_psivalues,sa_stack_score_psivalues,c_credo_score_psivalues,s_credo_score_psivalues,fu_credo_score_psivalues,r_credo_score_psivalues,gen_credo_score_psivalues
0,2024-06,s_apps_score,Training,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2024-07,s_apps_score,Monthly,0.034568,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2024-08,s_apps_score,Monthly,0.035108,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2024-09,s_apps_score,Monthly,0.045170,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2024-10,s_apps_score,Monthly,0.044685,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [56]:
sq = """
with base as 
(select 
  a.digitalLoanAccountId, 
  FORMAT_DATE('%Y-%m', a.ln_appln_submit_datetime) Application_month,
  FORMAT_DATE('%F', DATE_TRUNC(a.ln_appln_submit_datetime, WEEK(MONDAY))) as Appl_week_start_date,
  EXTRACT(WEEK(MONDAY) FROM a.ln_appln_submit_datetime) as Appl_week_number,
  a.ln_loan_type,
  case when date_trunc(a.ln_appln_submit_datetime, day) between '2023-07-01' and '2024-06-30' then 'Train'
       when date_trunc(a.ln_appln_submit_datetime, day) >= '2024-07-01' then 'Test'
       Else 'Other' end dataselection,
  a.s_cic_score,
 from 
 prj-prod-dataplatform.risk_mart.sil_risk_ds_master_20230101_20250206 a
 where a.ln_loan_applied_flag = 1 and ln_dl_rule_reject_flag = 0
 and date_trunc(a.ln_appln_submit_datetime, day) >= '2023-04-01'
 )
 select * from base where s_cic_score is not null and dataselection in ('Train', 'Test');"""
df = client.query(sq).to_dataframe(progress_bar_type='tqdm')

Job ID 47b9a622-2fb9-4f1d-ac23-e2d0d9435d50 successfully executed: 100%|██████████|
Downloading: 100%|██████████|


In [57]:
df.groupby(['dataselection'])['digitalLoanAccountId'].nunique()

dataselection
Test     88882
Train    31716
Name: digitalLoanAccountId, dtype: int64